In [26]:
from dateutil.relativedelta import relativedelta
import datetime
import numpy as np
import ee
ee.Initialize()

In [66]:
def define_image(image_id):
    le7_bands = [f"B{i}" for i in range(1, 8)] + ["B10", "B11"]
    im = ee.Image(f"LANDSAT/LE07/C01/T1_SR/{image_id}")
    elevation = ee.Image('CGIAR/SRTM90_V4')\
      .select("elevation")
        
    im = im.addBands(elevation)\
      .addBands(ee.Terrain.slope(elevation))\
      .toDouble()\
      .reproject(ee.Projection('EPSG:4326'), None, 30)
    
    date = im.getInfo()["properties"]["system:time_start"]
    date = datetime.datetime.fromtimestamp(date / 1000, datetime.timezone.utc)
    if date >= datetime.datetime(2003, 5, 31, tzinfo=datetime.timezone.utc):
        im = fill_image(im)

    return im


def fetch_list(l, folder_name="ee_export", fnames=None):
    if fnames is None:
        fnames = ["img"] * l.size().getInfo()
    
    tasks = []
    for i in range(l.size().getInfo()):
        task = ee.batch.Export.image.toDrive(
            ee.Image(l.get(i)),
            fnames[i],
            folder=folder_name
        )
        task.start()
        tasks.append(task)
    return tasks

In [76]:
def fill_image(img):
    fill = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR")\
             .filterDate("2000-01-01", "2000-12-31")\
             .filter(ee.Filter.eq("WRS_ROW", img.get("WRS_ROW")))\
             .filter(ee.Filter.eq("WRS_PATH", img.get("WRS_PATH")))
    return ee.Image(fill.sort("CLOUD_COVER").first()).toDouble()
                     
    
def gapfill(source, fill, kernel_size = 10, upscale = True):
    min_scale, max_scale = 1/3, 3
    min_neighbours = 64
    # Apply the USGS L7 Phase-2 Gap filling protocol, using a single kernel size.
    kernel = ee.Kernel.square(kernel_size * 30, "meters", False)
    # Find the pixels common to both scenes.
    common = source.mask().And(fill.mask())
    fc = fill.updateMask(common)
    sc = source.updateMask(common)
    # Find the primary scaling factors with a regression.
    # Interleave the bands for the regression.  This assumes the bands have the same names.
    regress = fc.addBands(sc)
    regress = regress.select(regress.bandNames().sort())
    ratio = 5
    if upscale:
        fit = regress.reduceResolution(ee.Reducer.median(), False, 500).reproject(regress.select(0).projection().scale(ratio, ratio)).reduceNeighborhood(ee.Reducer.linearFit().forEach(source.bandNames()), kernel, "kernel", False).unmask().reproject(regress.select(0).projection().scale(ratio, ratio))
    else:
        fit = regress.reduceNeighborhood(ee.Reducer.linearFit().forEach(source.bandNames()), kernel, "kernel", False)
    offset = fit.select(".*_offset")
    scale = fit.select(".*_scale")
    # Find the secondary scaling factors using just means and stddev
    reducer = ee.Reducer.mean().combine(ee.Reducer.stdDev(), "", True)
    if upscale:
        src_stats = source.reduceResolution(ee.Reducer.median(), False, 500).reproject(regress.select(0).projection().scale(ratio, ratio)).reduceNeighborhood(reducer, kernel, "kernel", False).reproject(regress.select(0).projection().scale(ratio, ratio))
        fill_stats = fill.reduceResolution(ee.Reducer.median(), False, 500).reproject(regress.select(0).projection().scale(ratio, ratio)).reduceNeighborhood(reducer, kernel, "kernel", False).reproject(regress.select(0).projection().scale(ratio, ratio))
    else:
        src_stats = source.reduceNeighborhood(reducer, kernel, "kernel", False)
        fill_stats = fill.reduceNeighborhood(reducer, kernel, "kernel", False)
    scale2 = src_stats.select(".*stdDev").divide(fill_stats.select(".*stdDev"))
    offset2 = src_stats.select(".*mean").subtract(fill_stats.select(".*mean").multiply(scale2))
    invalid = scale.lt(min_scale).Or(scale.gt(max_scale))
    scale = scale.where(invalid, scale2)
    offset = offset.where(invalid, offset2)
    # When all else fails, just use the difference of means as an offset.
    invalid2 = scale.lt(min_scale).Or(scale.gt(max_scale))
    scale = scale.where(invalid2, 1)
    offset = offset.where(invalid2, src_stats.select(".*mean").subtract(fill_stats.select(".*mean")))
    # Apply the scaling and mask off pixels that didn"t have enough neighbors.
    count = common.reduceNeighborhood(ee.Reducer.count(), kernel, "kernel", True)
    scaled = fill.multiply(scale).add(offset).updateMask(count.gte(min_neighbours))
    return source.unmask(scaled, True)

In [77]:
img_ids =[
    "LE07_152033_20060731",
    "LE07_152034_20060731",
    "LE07_152035_20060917",
    "LE07_153036_20060924",
    "LE07_151035_20060708",
    "LE07_151034_20050822",
    "LE07_150036_20050916",
    "LE07_150035_20050916",
    "LE07_150034_20050916",
    "LE07_149037_20041024",
    "LE07_149036_20071102",
    "LE07_149035_20070915",
    "LE07_149034_20060726",
    "LE07_148037_20071127",
    "LE07_148036_20050902",
    "LE07_148035_20061108",
    "LE07_147038_20040908",
    "LE07_147037_20060930",
    "LE07_147036_20060930",
    "LE07_147035_20050826",
    "LE07_146037_20071231",
    "LE07_146036_20090814",
    "LE07_146039_20051123",
    "LE07_146038_20060923",
    "LE07_145039_20011020",
    "LE07_144039_20011013",
    "LE07_143039_20081212",
    "LE07_143040_20051017",
    "LE07_140041_20071221",
    "LE07_139041_20071214",
    "LE07_138041_20071223",
    "LE07_137041_20060127",
    "LE07_136041_20060731",
    "LE07_136040_20081109",
    "LE07_135040_20081204",
    "LE07_134040_20090927"
]

In [78]:
img_list = ee.List([])
for im in img_list:
    img_list = img_list.add(define_image(im))
    
tasks = fetch_list(img_list, "glacier_images", img_ids)

In [79]:
tasks[0].status()

{'state': 'READY',
 'description': 'LE07_152033_20060731',
 'creation_timestamp_ms': 1620254986383,
 'update_timestamp_ms': 1620254986383,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'OQCNBMEAXQ5Q2QIXAROSAOBX',
 'name': 'projects/earthengine-legacy/operations/OQCNBMEAXQ5Q2QIXAROSAOBX'}